In [3]:
!pip install pyspark

  Using cached py4j-0.10.9-py2.py3-none-any.whl (198 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.2
    Uninstalling py4j-0.10.9.2:
      Successfully uninstalled py4j-0.10.9.2


In [4]:
!pip install reverse_geocoder

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,round,udf,lit
from pyspark.sql.types import *
import pyspark.sql.functions as F
from datetime import datetime
# import reverse_geocode as rg
import pytz

ModuleNotFoundError: No module named 'pyspark'

In [4]:
spark = SparkSession.builder.appName("Cluster").getOrCreate()
spark

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
path = '/content/drive/MyDrive/Temp/location.csv'
df = spark.read.csv(path,header=True, inferSchema= True)

In [7]:
df.show()

+----------+--------------------+--------+-----------+-----------+-------------------+--------+-----------------+---------------+---------------+-------+----------+--------------+----------+--------------+------+-------+-----------+-------+----+----+----+----+
| timestamp|                 aid|aid_type|   latitude|  longitude|horizontal_accuracy|altitude|altitude_accuracy|location_method|             ip|     OS|OS_version|  manufacturer|     model|       carrier|  _c15|   _c16|       _c17|   _c18|_c19|_c20|_c21|_c22|
+----------+--------------------+--------+-----------+-----------+-------------------+--------+-----------------+---------------+---------------+-------+----------+--------------+----------+--------------+------+-------+-----------+-------+----+----+----+----+
|1631664420|82c84ffe-525b-489...|    AAID|  13.063063|  77.501077|               24.7|       0|                0|           cell|   157.50.63.39|Android|        11|       Samsung|  SM-A125F|405-053;AirTel|  null|   nu

In [8]:
df.printSchema()

root
 |-- timestamp: integer (nullable = true)
 |-- aid: string (nullable = true)
 |-- aid_type: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- horizontal_accuracy: double (nullable = true)
 |-- altitude: integer (nullable = true)
 |-- altitude_accuracy: integer (nullable = true)
 |-- location_method: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- OS_version: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)



In [9]:
spark.conf.set("spark.sql.session.timeZone", "Asia/Kolkata")
data = df.select(['timestamp','aid','latitude','longitude'])
# data =data.withColumn("time", F.from_unixtime(col("timestamp"),"HH:MM"))


In [10]:
data.show()

+----------+--------------------+-----------+-----------+
| timestamp|                 aid|   latitude|  longitude|
+----------+--------------------+-----------+-----------+
|1631664420|82c84ffe-525b-489...|  13.063063|  77.501077|
|1631664420|d812f8e4-11e8-46a...|  10.823263|  78.679671|
|1631664420|99caf680-bef5-482...|  31.531531|  75.931872|
|1631664420|d6bd9f4c-8d53-4bc...|  14.684684|  77.616377|
|1631664420|8139ca2c-0dd7-421...|  17.441441|  78.434971|
|1631664420|e977134a-4182-4e2...|  12.972972|  80.098482|
|1631664420|92afe3fb-c6a7-409...|   16.45046|  76.688806|
|1631664420|6d24431b-313d-4aa...|21.00876808|75.54844665|
|1631664420|299857a9-eb9d-4fe...|  30.910903|   75.85321|
|1631664421|97293b20-9849-445...|   19.96822|  73.778857|
|1631664421|dba0826d-f1d2-43f...|  23.219756|  88.838095|
|1631664421|13243541-90f3-468...|18.47198677|73.85366058|
|1631664421|bfd150db-13bb-414...|   19.14679|  72.999833|
|1631664421|ecebcc80-b20c-4e8...|  11.387387|  75.927082|
|1631664421|c8

In [11]:
@udf(returnType=IntegerType())
def conv2time(x):
    ist = pytz.timezone("Asia/Kolkata")
    date = datetime.fromtimestamp(x, ist)
    # print(date.strftime("%H"))
    return int(date.strftime("%H"))

In [12]:
# data = data.withColumn("time", F.from_unixtime("timestamp","HH:MM:SS"))

In [13]:
data = data.withColumn("time", conv2time(col("timestamp")))

In [14]:
data.show()

+----------+--------------------+-----------+-----------+----+
| timestamp|                 aid|   latitude|  longitude|time|
+----------+--------------------+-----------+-----------+----+
|1631664420|82c84ffe-525b-489...|  13.063063|  77.501077|   5|
|1631664420|d812f8e4-11e8-46a...|  10.823263|  78.679671|   5|
|1631664420|99caf680-bef5-482...|  31.531531|  75.931872|   5|
|1631664420|d6bd9f4c-8d53-4bc...|  14.684684|  77.616377|   5|
|1631664420|8139ca2c-0dd7-421...|  17.441441|  78.434971|   5|
|1631664420|e977134a-4182-4e2...|  12.972972|  80.098482|   5|
|1631664420|92afe3fb-c6a7-409...|   16.45046|  76.688806|   5|
|1631664420|6d24431b-313d-4aa...|21.00876808|75.54844665|   5|
|1631664420|299857a9-eb9d-4fe...|  30.910903|   75.85321|   5|
|1631664421|97293b20-9849-445...|   19.96822|  73.778857|   5|
|1631664421|dba0826d-f1d2-43f...|  23.219756|  88.838095|   5|
|1631664421|13243541-90f3-468...|18.47198677|73.85366058|   5|
|1631664421|bfd150db-13bb-414...|   19.14679|  72.99983

In [15]:
data.printSchema()

root
 |-- timestamp: integer (nullable = true)
 |-- aid: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- time: integer (nullable = true)



In [16]:
import reverse_geocoder as rg
coordinates = (-37.81, 144.96), (31.76, 35.21)
result = rg.search(coordinates)

Loading formatted geocoded file...


In [17]:
result[0]

OrderedDict([('lat', '-37.814'),
             ('lon', '144.96332'),
             ('name', 'Melbourne'),
             ('admin1', 'Victoria'),
             ('admin2', 'Melbourne'),
             ('cc', 'AU')])

In [18]:
# @udf(returnType=ArrayType(StringType()))
@udf(returnType=StringType())
def coordinates_to_city_name(latitude,longitude):
  res = rg.search((latitude,longitude))
  return res[0]['name']


In [19]:
@udf(returnType=StringType())
def coordinates_to_dist_name(latitude,longitude):
  res = rg.search((latitude,longitude))
  return res[0]['admin2']

In [20]:
coordinates_to_city_name(lit(-37.81),lit(144.96))

Column<'coordinates_to_city_name(-37.81, 144.96)'>

In [21]:
coordinates_to_dist_name(lit(-37.81),lit(144.96))

Column<'coordinates_to_dist_name(-37.81, 144.96)'>

In [22]:
# data = data.withColumn("city", coordinates_to_city_name(col('latitude'),col('longitude')))

In [23]:
df_temp,_ = data.randomSplit([0.01, 0.99],seed=1234)

In [24]:
df_temp.count()

6817

In [25]:
temp = spark.createDataFrame([], StructType([]))

In [30]:
temp.withColumn(coordinates_to_city_name(lit(-37.81),lit(144.96)).alias("Cureated")).show(truncate=False)

KeyboardInterrupt: ignored

In [28]:
# df_temp.withColumn("Cureated", coordinates_to_city_name(lit(-37.81),lit(144.96))).show(truncate=False)

KeyboardInterrupt: ignored